<img align="right" width="200" height="200" src="https://static.tildacdn.com/tild6236-6337-4339-b337-313363643735/new_logo.png">

# Spark Structured Streaming I
**Сергей Гришаев**  
serg.grishaev@gmail.com  

## На этом занятии
+ Общие сведения
+ Rate streaming
+ File streaming
+ Kafka streaming

## Общие сведения

Системы поточной обработки данных:
- работают с непрерывным потоком данных
- нужно хранить состояние стрима
- результат обработки быстро появляется в целевой системе
- должны проектироваться с учетом требований к высокой доступности
- важная скорость обработки данных и время зажержки (лаг)

### Примеры систем поточной обработки данных

#### Карточный процессинг
- нельзя терять платежи
- нельзя дублировать платежи
- простой сервиса недопустим
- максимальное время задержки ~ 1 сек
- небольшой поток событий
- OLTP

#### Обработка логов 
- потеря единичных событий допустима
- дублирование единичных событий допустимо
- простой сервиса допустим
- максимальное время задержки ~ 1 час
- большой поток событий
- OLAP

### Виды стриминг систем

#### Real-time streaming
- низкие задержки на обработку
- низкая пропускная способность
- подходят для критичных систем
- пособытийная обработка
- OLTP
- exactly once consistency (нет потери данных и нет дубликатов)

#### Micro batch streaming
- высокие задержки
- высокая пропускная способность
- не подходят для критичных систем
- обработка батчами
- OLAP
- at least once consistency (во время сбоев могут возникать дубликаты)

### Выводы:
+ Существуют два типа систем поточной обработки данных - real-time и micro-batch
+ Spark Structured Streaming является micro-batch системой
+ При работе с большими данными обычно пропускная способность важнее, чем время задержки


## Rate streaming

Самый простой способ создать стрим - использовать `rate` источник. Созданный DF является streaming, о чем нам говорит метод создания `readStream` и атрибут `isStreaming`. `rate` хорошо подходит для тестирования приложений, когда нет возможности подключится к потоку реальных данных

In [ ]:
val df = spark.range(10)

In [ ]:
df.isStreaming

In [ ]:
val sdf = spark.readStream.format("rate").load

In [ ]:
sdf.isStreaming

У `sdf`, как и у любого DF, есть схема и план выполнения:

In [ ]:
sdf.printSchema
sdf.explain(true)

В отличии от обычных DF, у `sdf` нет таких методов, как `show`, `collect`, `take`. Для них также недоступен Dataset API. Поэтому для того, чтобы посмотреть их содержимое, мы должны использовать `console` синк и создать `StreamingQuery`. Процессинг начинается только после вызова метода `start`. `trigger` позволяет настроить, как часто стрим будет читать новые данные и обрабатывать их

In [27]:
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.DataFrame

def createConsoleSink(df: DataFrame) = {
    df
    .writeStream
    .format("console")
    .trigger(Trigger.ProcessingTime("10 seconds"))
    .option("truncate", "false")
    .option("numRows", "20")
}

createConsoleSink: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.streaming.DataStreamWriter[org.apache.spark.sql.Row]


In [ ]:
val sink = createConsoleSink(sdf)

In [ ]:
val sq = sink.start // StreamingQuery

Чтобы остановить DF, можно вызвать метод `stop` к `sdf`, либо получить список всех streming DF и остановить их:

In [ ]:
sq.stop

In [24]:
import org.apache.spark.sql.SparkSession

def killAll() = {
    SparkSession
        .active
        .streams
        .active
        .foreach { x =>
                    val desc = x.lastProgress.sources.head.description
                    x.stop
                    println(s"Stopped ${desc}")
        }               
}

killAll: ()Unit


In [ ]:
killAll()

Создадим стрим, выполняющий запись в `parquet` файл:

In [ ]:
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.DataFrame

def createParquetSink(df: DataFrame, 
                      fileName: String) = {
    df
    .repartition(1)
    .writeStream
    .format("parquet")
    .option("path", s"/tmp/spark-intro-stream/$fileName")
    .option("checkpointLocation", s"/tmp/chk_01/$fileName")
    .trigger(Trigger.ProcessingTime("10 seconds"))
}

In [ ]:
import sys.process._

In [ ]:
"""hadoop fs -mkdir /tmp/spark-intro-stream """.!

In [ ]:
"""hadoop fs -ls / """.!!

In [ ]:
val sink = createParquetSink(sdf, "s1.parquet")

In [ ]:
val sq = sink.start

In [ ]:
Убедимся, что стрим пишется в файл:

In [ ]:
import sys.process._
"hadoop fs -ls  /tmp/spark-intro-stream/s1.parquet/*".!

Прочитаем файл с помощью Spark:

In [ ]:
val rates = spark.read.parquet("/tmp/spark-intro-stream/s1.parquet")
println(rates.count)
rates.printSchema
rates.show(5, false)

Параллельно внутри одного Spark приложения может работать несколько стримов:

In [ ]:
val consoleSink = createConsoleSink(sdf)
val consoleSq = consoleSink.start

In [ ]:
killAll

Напишем функцию, которая добавляет к нашей колонке случайный `ident` аэропорта из датасета [Airport Codes](https://datahub.io/core/airport-codes)  

In [ ]:
val csvOptions = Map("header" -> "true", "inferSchema" -> "true")
val airports = spark.read.options(csvOptions).csv("/tmp/airport-codes.csv")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

In [ ]:
val idents = airports.select('ident).limit(200).distinct.as[String].collect

In [ ]:
import org.apache.spark.sql.functions._
val identSdf = sdf.withColumn("ident", shuffle(array(idents.map(lit(_)):_*))(0))

In [ ]:
val identPqSink = createParquetSink(identSdf, "s2.parquet")
val identPqSq = identPqSink.start

Проверим, что данные записываются в `parquet`

In [ ]:
val identPq = spark.read.parquet("/tmp/spark-intro-stream/s2.parquet")
println(identPq.count)
identPq.printSchema
identPq.show(5, false)

Временно остановим стрим, он понадобится нам для следующих экспериментов

In [ ]:
killAll

### Выводы:
- `rate` - самый простой способ создать стрим для тестирования приложений
- стрим начинает работу после вызова метода `start` и не блокирует основной поток программы
- в одном Spark приложении может работать несколько стримов одновременно

## File Streaming
Spark позволяет запустить стрим, который будет "слушать" директорию и читать из нее новые файлы. При этом за раз будет прочитано количество файлов, установленное в параметре `maxFilesPerTrigger` [ссылка](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#input-sources). В этом кроется одна из основных проблем данного источника. Поскольку стрим, сконфигурированный под чтение небольших файлов, может "упасть", если в директорию начнут попадать файлы большого объема. Создадим стрим из директории `datasets/s2.parquet`:

In [ ]:
val sdfFromParquet = spark
        .readStream
        .format("parquet")
        .option("maxFilesPerTrigger", "1")
        .option("path", "/tmp/spark-intro-stream/s2.parquet")
        .load

sdfFromParquet.printSchema

Поскольку в директорию могут попасть любые данные, а df должен иметь фиксированную схему, то Spark не позволяет нам создавать SDF на основе файлов без указания схемы.

In [ ]:
import org.apache.spark.sql.functions._

val letters = List("a", "b", "c", "d", "e", "f", "g", "i")
val condition = letters.map { x => col("ident").startsWith(x) }.reduce { (x,y) => x or y }

val sdfFromParquet = spark
        .readStream
        .format("parquet")
        .schema(identPq.schema)
        .option("maxFilesPerTrigger", "1")
        .option("path", "/tmp/spark-intro-stream/s2.parquet")
        .load
        .withColumn("ident", lower('ident))

sdfFromParquet.printSchema

In [ ]:
val opts = Map(
    "maxFilesPerTrigger" -> "100", 
    "path" -> "/tmp/spark-intro-stream/s2.parquet"
)

val sdfFromParquet = spark
        .readStream
        .format("parquet")
        .schema(identPq.schema)
        .options(opts)
        .load
        .withColumn("ident", lower('ident))

In [ ]:
val consoleSink = createConsoleSink(sdfFromParquet)
consoleSink.start

In [ ]:
killAll

File source позволяет со всеми типами файлов, с которыми умеет работать Spark: `parquet`, `orc`, `csv`, `json`, `text`.

### Выводы:
- Spark позволяет создавать SDF на базе всех поддерживаемых типов файлов
- При создании SDF вы должны указать схему данных
- File streaming имеет несколько серьезных недостатков:
  + Входной поток можно ограничить только макисмальным количество файлов, попадающих в батч
  + Если стрим упадает посередине файла, то при перезапуске эти данные будут обработаны еще раз

<img align="right" width="100" height="100" src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Apache_kafka.svg/1200px-Apache_kafka.svg.png">

## Kafka streaming

https://kafka.apache.org

**Apache Kafka** - самая распространенная в мире система, на основе которой строятся приложения для поточной обработки данных. Она имеет несколько преимуществ:
- высокая пропускная способность
- высокая доступность за счет распределенной архитектуры и репликации
- у каждого сообщения есть свой номер, который называется offset, что позволяет гранулярно сохранять состояние стрима

### Архитектура системы

#### Topic
Топик - это таблицы в Kafka. Мы пишем данные в топик и читаем данные из топика. Топик как правило распределен по нескольким узлам кластера для обеспечения высокой доступности и скорости работы с данными

<img align="center" width="500" height="500" src="https://kafka.apache.org/25/images/log_anatomy.png">

#### Partition
Партиции - это блоки, из которых состоят топики. Партиция представляет собой неделимый блок, который хранится на одном из узлов. Топик может иметь произвольное количество партиций. Чем больше партиций - тем выше параллелзим при чтении и записи, однако слишком большое число партиций в топике может привести к замедлению работы всей системы.

#### Replica
Каждая партиция имеет (может иметь) несколько реплик. Внешние приложения всегда работают (читают и пишут) с основной репликой. Остальные реплики являются дочерними и не используются во внешнем IO. Если узел, на котором расположена основная реплика, падает, то одна из дочерних реплик становится основной и работа с данными продолжается

#### Message
Сообщения - это данные, которые мы пишем и читаем в Kafka. Они представлены кортежем (Key, Value), но ключ может быть иметь значение `null` (используется не всегда). Сереализация и десереализация данных всегда происходит на уровне клиентов Kafka. Сама Kafka ничего о типах данных не знает и хранит ключи и значения в виде массива байт

#### Offset
Оффсет - это порядковый номер сообщения в партиции. Когда мы пишем сообщение (сообщение всегда пишется в одну из партиций топика), Kafka помещает его в топик с номер `n+1`, где `n` - номер последнего сообщения в этом топике

<img align="center" width="400" height="400" src="https://kafka.apache.org/25/images/log_consumer.png">

#### Producer
Producer - это приложение, которое пишет в топик. Producer'ов может быть много. Параллельная запись достигается за счет того, что каждое новое сообщение попадает в случайную партицию топика (если не указан `key`)

#### Consumer
Consumer - это приложение, читающее данные из топика. Consumer'ов может быть много, в этом случае они называются `consumer group`. Параллельное чтение достигается за счет распределения партиций топика между consumer'ами в рамках одной группы. Каждый consumer читает данные из "своих" партиций и ничего про другие не знает. Если consumer падает, то "его" партиции переходят другим consumer'ам.

#### Commit
Коммитом в Kafka называют сохранение информации о факте обработки сообщения с определенным оффсетом. Поскольку оффсеты для каждой партиции топика свои, то и информация о последнем обработанном оффсете хранится по каждой партиции отдельно. Обычные приложения пишут коммиты в специальный топик Kafka, который имеет название `__consumer_offsets`. Spark хранит обработанные оффсеты по каждому батчу в ФС (например, в HDFS).

#### Retention
Поскольку кластер Kafka не может хранить данные вечно, то в ее конфигурации задаются пороговые значение по **объему** и **времени хранения** для каждого топика, при превышении которых данные удаляются. Например, если у топика A установлен renention по времени 1 месяц, то данные будут хранится в системе не менее одного месяца (и затем будут удалены одной из внутренних подсистем)

### Spark connector
https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10  
https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html  

### Запуск Kafka в docker
```shell
docker run --rm \
   -p 2181:2181 \
   --name=test_zoo \
   -e ZOOKEEPER_CLIENT_PORT=2181 \
   confluentinc/cp-zookeeper
```

```shell
docker run --rm \
    -p 9092:9092 \
    --name=test_kafka \
    -e KAFKA_ZOOKEEPER_CONNECT=host.docker.internal:2181 \
    -e KAFKA_ADVERTISED_LISTENERS=PLAINTEXT://host.docker.internal:9092 \
    -e KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR=1 \
    confluentinc/cp-kafka
```

### Работа с Kafka с помощь Static Dataframe

Spark позволяет работать с кафкой как с обычной базой данных. Запишем данные в топик `test_topic0`. Для этого нам необходимо подготовить DF, в котором будет две колонки:
- `value: String` - данные, которые мы хотим записать
- `topic: String` - топик, куда писать каждую строку DF

In [8]:
val identPq = spark.read.parquet("/tmp/spark-intro-stream/s2.parquet")
println(identPq.count)
identPq.printSchema
identPq.show(5, false)

1678
root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)
 |-- ident: string (nullable = true)

+-----------------------+-----+-----+
|timestamp              |value|ident|
+-----------------------+-----+-----+
|2023-07-04 19:59:39.598|38   |02OI |
|2023-07-04 19:59:41.598|40   |01PA |
|2023-07-04 19:59:43.598|42   |02KY |
|2023-07-04 19:59:45.598|44   |01FA |
|2023-07-04 19:59:47.598|46   |01PN |
+-----------------------+-----+-----+
only showing top 5 rows



identPq = [timestamp: timestamp, value: bigint ... 1 more field]


[timestamp: timestamp, value: bigint ... 1 more field]

In [ ]:
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

def writeKafka[T](topic: String, data: Dataset[T]): Unit = {
    val kafkaParams = Map(
        "kafka.bootstrap.servers" -> "spark-master-1.newprolab.com:6667"
    )
    
    val output = data.select(to_json(struct(col("*"))).alias("value"))
    output.printSchema
    output.show(2, false)
    output.withColumn("key", lit("hello world"))
            .withColumn("topic", lit(topic)).write.format("kafka").options(kafkaParams).save
}

In [ ]:
writeKafka("test_topic3", identPq)

Прочитаем данные из Kafka:

In [3]:
import org.apache.spark.sql.functions._

In [5]:
val kafkaParams = Map(
        "kafka.bootstrap.servers" -> "spark-master-1.newprolab.com:6667",
        "subscribe" -> "test_topic3"
    )


val df = spark.read.format("kafka").options(kafkaParams).load

df.printSchema
df.show(3, 30)
df.select(col("key").cast("string"), col("value").cast("string")).show(3, false)

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

+------------------------------+------------------------------+-----------+---------+------+-----------------------+-------------+
|                           key|                         value|      topic|partition|offset|              timestamp|timestampType|
+------------------------------+------------------------------+-----------+---------+------+-----------------------+-------------+
|[68 65 6C 6C 6F 20 77 6F 72...|[7B 22 74 69 6D 65 73 74 61...|test_topic3|        0|     0|2023-07-04 21:07:37.149|            0|
|[68 65 6C 6C 6F 20 77 6F 72...|[7B 22 74 69 6D 65 73 74 61...|test_topic3|        0|     1|2023-07-04 21:07:37.161|            0|
|[68 65 6C 6C 6F 20 77 6F 72...|[7B 22 74 69 6D 65 73 74 61...

kafkaParams = Map(kafka.bootstrap.servers -> spark-master-1.newprolab.com:6667, subscribe -> test_topic3)
df = [key: binary, value: binary ... 5 more fields]


[key: binary, value: binary ... 5 more fields]

In [6]:
df.rdd.partitions.foreach(println)

KafkaSourceRDDPartition(0,KafkaSourceRDDOffsetRange(test_topic3-0,-2,-1,None))


In [7]:
df.select(col("value").cast("string")).show(3, false)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|{"timestamp":"2023-07-04T19:59:39.598+03:00","value":38,"ident":"02OI"}|
|{"timestamp":"2023-07-04T19:59:41.598+03:00","value":40,"ident":"01PA"}|
|{"timestamp":"2023-07-04T19:59:43.598+03:00","value":42,"ident":"02KY"}|
+-----------------------------------------------------------------------+
only showing top 3 rows



In [9]:
df
.select(from_json(col("value").cast("string"), identPq.schema).alias("value"))
.show(3, false)

+-----------------------------------+
|value                              |
+-----------------------------------+
|[2023-07-04 19:59:39.598, 38, 02OI]|
|[2023-07-04 19:59:41.598, 40, 01PA]|
|[2023-07-04 19:59:43.598, 42, 02KY]|
+-----------------------------------+
only showing top 3 rows



In [12]:
df
.select(from_json(col("value").cast("string"), identPq.schema).alias("value"))
.select(col("value.*")).show(3, false)

+-----------------------+-----+-----+
|timestamp              |value|ident|
+-----------------------+-----+-----+
|2023-07-04 19:59:39.598|38   |02OI |
|2023-07-04 19:59:41.598|40   |01PA |
|2023-07-04 19:59:43.598|42   |02KY |
+-----------------------+-----+-----+
only showing top 3 rows



Чтение из Kafka имеет несколько особенностей:
- по умолчанию читается все содержимое топика. Поскольку обычно в нем много данных, эта операция может создать большую нагрузку на кластер Kafka и Spark приложение
- колонки `value` и `key` имеют тип `binary`, который необходимо десереализовать

Чтобы прочитать только определенную часть топика, нам необходимо задать минимальный и максимальный оффсет для чтения с помощью параметров `startingOffsets` , `endingOffsets`. Возьмем два случайных события:

In [16]:
df.sample(0.1).limit(2).select('topic, 'partition, 'offset).show

+-----------+---------+------+
|      topic|partition|offset|
+-----------+---------+------+
|test_topic3|        0|     0|
|test_topic3|        0|    16|
+-----------+---------+------+



На основании этих событий подготовим параметры `startingOffsets` и `endingOffsets`

In [20]:
val kafkaParams = Map(
        "kafka.bootstrap.servers" -> "spark-master-1.newprolab.com:6667",
        "subscribe" -> "test_topic3",
        "startingOffsets" -> """{"test_topic3":{"0": 2}}""",
        "endingOffsets" -> """{"test_topic3":{"0": 500}}"""
    )


val df = spark.read.format("kafka").options(kafkaParams).load

df.printSchema
df.show(20)

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

+--------------------+--------------------+-----------+---------+------+--------------------+-------------+
|                 key|               value|      topic|partition|offset|           timestamp|timestampType|
+--------------------+--------------------+-----------+---------+------+--------------------+-------------+
|[68 65 6C 6C 6F 2...|[7B 22 74 69 6D 6...|test_topic3|        0|     2|2023-07-04 21:07:...|            0|
|[68 65 6C 6C 6F 2...|[7B 22 74 69 6D 6...|test_topic3|        0|     3|2023-07-04 21:07:...|            0|
|[68 65 6C 6C 6F 2...|[7B 22 74 69 6D 6...|test_topic3|        0|     4|2023-07-04 21:07:...|            0|
|[68 65 6C 6C 6F 2...|[7B 22 74 69 6D 6...|test_topic3|        0|    

kafkaParams = Map(kafka.bootstrap.servers -> spark-master-1.newprolab.com:6667, subscribe -> test_topic3, startingOffsets -> {"test_topic3":{"0": 2}}, endingOffsets -> {"test_topic3":{"0": 500}})
df = [key: binary, value: binary ... 5 more fields]


[key: binary, value: binary ... 5 more fields]

In [ ]:
// latest/earliest - offset options

По умолчанию параметр `startingOffsets` имеет значение `earliest`, а `endingOffsets` - `latest`. Поэтому, когда мы не указывали эти параметры, Spark прочитал содержимое всего топика

Чтобы получить наши данные, которые мы записали в топик, нам необходимо их десереализовать. В нашем случае достаточно использовать `.cast("string")`, однако это работает не всегда, т.к. формат данных может быть произвольным.

In [21]:
val jsonString = df.select('value.cast("string")).as[String]

jsonString.show(20, false)

val parsed = spark.read.json(jsonString)
parsed.printSchema
parsed.show(20, false)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|{"timestamp":"2023-07-04T19:59:43.598+03:00","value":42,"ident":"02KY"}|
|{"timestamp":"2023-07-04T19:59:45.598+03:00","value":44,"ident":"01FA"}|
|{"timestamp":"2023-07-04T19:59:47.598+03:00","value":46,"ident":"01PN"}|
|{"timestamp":"2023-07-04T19:59:49.598+03:00","value":48,"ident":"02PS"}|
|{"timestamp":"2023-07-04T19:59:51.598+03:00","value":50,"ident":"02IN"}|
|{"timestamp":"2023-07-04T19:59:53.598+03:00","value":52,"ident":"02FL"}|
|{"timestamp":"2023-07-04T19:59:55.598+03:00","value":54,"ident":"02PS"}|
|{"timestamp":"2023-07-04T19:59:57.598+03:00","value":56,"ident":"03CO"}|
|{"timestamp":"2023-07-04T19:59:59.598+03:00","value":58,"ident":"02MI"}|
|{"timestamp":"2023-07-04T20:00:01.598+03:00","value":60,"ident":"02MD"}|
|{"timestamp":"2023-07-04T20:00:03.598

jsonString = [value: string]
parsed = [ident: string, timestamp: string ... 1 more field]


[ident: string, timestamp: string ... 1 more field]

### Работа с Kafka с помощью Streaming DF
При создании SDF из Kafka необходимо помнить, что:
- `startingOffsets` по умолчанию имеет значение `latest`
- `endingOffsets` использовать нельзя
- количество сообщений за батч можно (и нужно) ограничить параметром `maxOffsetPerTrigger` (по умолчанию он не задан и первый батч будет содержать данные всего топика

In [36]:
val kafkaParams = Map(
        "kafka.bootstrap.servers" -> "spark-master-1.newprolab.com:6667",
        "subscribe" -> "test_topic3",
        "startingOffsets" -> """earliest""",
        "maxOffsetsPerTrigger" -> "50",
        "failOnDataLoss" -> "false"
    )

val sdf = spark.readStream.format("kafka").options(kafkaParams).load
val parsedSdf = sdf.select('value.cast("string"), 'topic, 'partition, 'offset)

val outputDf = parsedSdf.select(
                            get_json_object(col("value"), "$.ident").alias("ident"),
                            get_json_object(col("value"), "$.value").alias("value")
                        )
                        .groupBy(col("ident")).agg(avg(col("value")))

outputDf
    .writeStream
    .format("console")
    .outputMode("append")
    .option("numRows", "30")
    .option("truncate", "false")
    .start()

//val sink = createConsoleSink(parsedSdf)

//val sq = sink.start

Name: org.apache.spark.sql.AnalysisException
Message: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;;
Aggregate [ident#1522], [ident#1522, avg(cast(value#1523 as double)) AS avg(value)#1529]
+- Project [get_json_object(value#1517, $.ident) AS ident#1522, get_json_object(value#1517, $.value) AS value#1523]
   +- Project [cast(value#1504 as string) AS value#1517, topic#1505, partition#1506, offset#1507L]
      +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@2cb51d17, kafka, Map(maxOffsetsPerTrigger -> 5, startingOffsets -> earliest, subscribe -> test_topic3, kafka.bootstrap.servers -> spark-master-1.newprolab.com:6667), [key#1503, value#1504, topic#1505, partition#1506, offset#1507L, timestamp#1508, timestampType#1509], StreamingRelation DataSource(org.apache.spark.sql.SparkSession@42b753fa,kafka,List(),None,List(),None,Map(maxOffsetsPerTrigger -> 5, startingOffsets -> earliest, subscribe

In [37]:
killAll

lastException: Throwable = null


Если мы перезапустим этот стрим, он повторно прочитает все данные. Чтобы обеспечить сохранение состояния стрима после обработки каждого батча, нам необходимо добавить параметр `checkpointLocation` в опции `writeStream`:

In [38]:
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.DataFrame

def createConsoleSinkWithCheckpoint(chkName: String, df: DataFrame) = {
    df
    .writeStream
    .format("console")
    .trigger(Trigger.ProcessingTime("3 seconds"))
    .option("checkpointLocation", s"/tmp/chk_01/$chkName")
    .option("truncate", "false")
    .option("numRows", "20")
}

createConsoleSinkWithCheckpoint: (chkName: String, df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.streaming.DataStreamWriter[org.apache.spark.sql.Row]


In [43]:
val sink = createConsoleSinkWithCheckpoint("test1", parsedSdf)
val sq = sink.start

sink = org.apache.spark.sql.streaming.DataStreamWriter@46206936
sq = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@77e5e6a7


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@77e5e6a7

-------------------------------------------
Batch: 9
-------------------------------------------
+------------------------------------------------------------------------+-----------+---------+------+
|value                                                                   |topic      |partition|offset|
+------------------------------------------------------------------------+-----------+---------+------+
|{"timestamp":"2023-07-04T20:01:09.598+03:00","value":128,"ident":"01MI"}|test_topic3|0        |45    |
|{"timestamp":"2023-07-04T20:01:11.598+03:00","value":130,"ident":"00UT"}|test_topic3|0        |46    |
|{"timestamp":"2023-07-04T20:01:13.598+03:00","value":132,"ident":"02CO"}|test_topic3|0        |47    |
|{"timestamp":"2023-07-04T20:01:15.598+03:00","value":134,"ident":"01II"}|test_topic3|0        |48    |
|{"timestamp":"2023-07-04T20:01:17.598+03:00","value":136,"ident":"00XS"}|test_topic3|0        |49    |
+----------------------------------------------------------------------

In [44]:
killAll

Stopped KafkaV2[Subscribe[test_topic3]]


In [46]:
import sys.process._

In [47]:
"hadoop fs -ls /tmp/chk_01/test1".!!

"Found 4 items
drwxr-xr-x   - teacher hdfs          0 2023-07-04 21:39 /tmp/chk_01/test1/commits
-rw-r--r--   3 teacher hdfs         45 2023-07-04 21:38 /tmp/chk_01/test1/metadata
drwxr-xr-x   - teacher hdfs          0 2023-07-04 21:39 /tmp/chk_01/test1/offsets
drwxr-xr-x   - teacher hdfs          0 2023-07-04 21:38 /tmp/chk_01/test1/sources
"


In [59]:
"hadoop fs -ls /tmp/chk_01/test1/offsets/*".!

-rw-r--r--   3 teacher hdfs        430 2023-07-04 21:39 /tmp/chk_01/test1/offsets/0
-rw-r--r--   3 teacher hdfs        431 2023-07-04 21:39 /tmp/chk_01/test1/offsets/1
-rw-r--r--   3 teacher hdfs        431 2023-07-04 21:39 /tmp/chk_01/test1/offsets/10
-rw-r--r--   3 teacher hdfs        431 2023-07-04 21:39 /tmp/chk_01/test1/offsets/11
-rw-r--r--   3 teacher hdfs        431 2023-07-04 21:39 /tmp/chk_01/test1/offsets/12
-rw-r--r--   3 teacher hdfs        431 2023-07-04 21:39 /tmp/chk_01/test1/offsets/13
-rw-r--r--   3 teacher hdfs        431 2023-07-04 21:39 /tmp/chk_01/test1/offsets/14
-rw-r--r--   3 teacher hdfs        431 2023-07-04 21:39 /tmp/chk_01/test1/offsets/2
-rw-r--r--   3 teacher hdfs        431 2023-07-04 21:39 /tmp/chk_01/test1/offsets/3
-rw-r--r--   3 teacher hdfs        431 2023-07-04 21:39 /tmp/chk_01/test1/offsets/4
-rw-r--r--   3 teacher hdfs        431 2023-07-04 21:39 /tmp/chk_01/test1/offsets/5
-rw-r--r--   3 teacher hdfs        431 2023-07-04 21:39 /tmp/chk_01/tes

0

In [57]:
"hadoop fs -cat /tmp/chk_01/test1/offsets/12".!!

"v1
{"batchWatermarkMs":0,"batchTimestampMs":1688495979001,"conf":{"spark.sql.streaming.stateStore.providerClass":"org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider","spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion":"2","spark.sql.streaming.multipleWatermarkPolicy":"min","spark.sql.streaming.aggregation.stateFormatVersion":"2","spark.sql.shuffle.partitions":"200"}}
{"test_topic3":{"0":65}}
"


### Выводы:
- Apache Kafka - распределенная система, обеспечивающая передачу потока данных в слабосвязанных системах
- Работать с Kafka можно как с использованием Static DF, так и с помощью Streaming DF
- Чтобы стрим запоминал свое состояние после остановки, необходимо использовать checkpoint - директорию на HDFS (или локальной ФС), в которую будет сохранятся состояние стрима после каждого батча

В конце работы не забудьте остановить Spark:

In [ ]:
spark.stop

In [ ]:
sq.status

In [ ]:
println(sq.lastProgress)

In [ ]:
sq.awaitTermination(10000)